Preliminary Note: Recall that I cannot use any specifically markdown formatting in the notebook, since opening it in PyCharm (a necessary operation to ensure it has saved appropriately) will delete any cell with special markdown formatting.

Vision Revision:
For my final project, I propose to build a neural network trained to generate poetry in the style of John  Milton. My dataset will be a simple text file containing the entirety of Paradise Lost.

Background:
This project will use an LSTM (Long short-term memory) trained on the text of Paradise Lost. An LSTM is a specific type of recurrent neural network (RNN) which is structured specifically to be able to learn longer term interdependencies between inputs. This LSTM will analyze the text of Paradise Lost tokenized by letter, and will generate output coorespondingly. My model for this will be Chollet (https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/8.1-text-generation-with-lstm.ipynb) with some cross-referencing of another example (https://github.com/pranjal52/text_generators) which was unfortunately running too slow to be very useful.

Implementation:
My implementation is not significantly different from Chollet's at this point. Once it is, I will have more to say on this matter.

Results:
Again, since my implementation is about the same as Chollet's currently, I do not have final results yet. That being said, these preliminary results are very promising, I think, and there is only room for improvement from here with finetuning Chollet's model.

Implications:
Poetry is weird. A conglomeration of semi-coherent, half-connected words and phrases whose structure and interplay somehow amount to far more than their mere parts -- it does not seem like the sort of thing that a NN or AI (at least currently) could convincingly fake. It's too human, too subjective. But the fact that it is subjective and often so open to interpretation is exactly what makes the potential for NN-generated poetry so great. If someone doesn't know that a poem has been generated by an un-thinking AI (and no AI that I am aware of can currently be described as thinking, in a human sense), they will impute their own interpretations and understandings into the poem and attribute them to the author. If a NN can generate convingingly human poetry, it is due in large part to how nebulous and varying poetry is in the first place. But that fact is simultaneously why it would be difficult to train a NN to generate convincing poetry. The variation and inconsistency that makes poetry such a uniquely human affair also makes it difficult to train a NN on. This is one of the reasons why I have trouble believing any AI will ever be up to or exceed the capacity of a human beings in their own domains (obviously computers can easily surpass human beings at, say, raw computational power and things of that sort). There is something about us that makes us and our cultural activities distinct from mere mathematics.

In [23]:
import keras
import numpy as np

text = open("projectData.txt").read().lower()
print('Corpus length:', len(text))

Corpus length: 454186


In [24]:
# Length of extracted character sequences
maxlen = 120

# We sample a new sequence every `step` characters
step = 3

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 151356
Unique characters: 39
Vectorization...


In [30]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars)), return_sequences=True))
model.add(layers.Dropout(0.2))
model.add(layers.LSTM(128))
model.add(layers.Dense(len(chars), activation='softmax'))
print("Done")

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Done


In [31]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
print("Done")

Done


In [32]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

print("Done")

Done


In [33]:
import random
import sys

for epoch in range(1, 2):
    print('epoch', epoch)
    # Fit the model for 5 epochs on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
Epoch 1/1
151356/151356 [==============================] - 268s 2ms/step - loss: 2.1437
--- Generating with seed: "resh flourets hill and valley smil'd.
this saw his hapless foes, but stood obdur'd,
and to rebellious fight rallied thir"
------ temperature: 0.2
resh flourets hill and valley smil'd.
this saw his hapless foes, but stood obdur'd,
and to rebellious fight rallied thir sead to serve to the seate and the seast the serve the seemest the serve to seem the seeps to seeps to perve such to seers, and and to the serve to the fore the serve seep
to seem to neent with to with the serpent and the serve to her seeps and all the seeps to seep and the serve to when the serve of the seace to he wores and so serve the frough to the seard and to the serve serve the serve with 
------ temperature: 0.5
e to when the serve of the seace to he wores and so serve the frough to the seard and to the serve serve the serve with earth now levers searse sof reeuse.
 to seat to deer, and to soeds